# Insurance Challenge 2 - Baseline Submission

This notebook provides a simple baseline for **Insurance Challenge 2: Risk-Based Pricing**.

**Goal**: Predict `NextYearLoss` for each policy
**Metric**: Normalized Gini - Higher is better

## Instructions:
1. **Replace API credentials** in the first cell with your team's API key and name
2. **Run all cells** to generate and submit baseline predictions
3. **Check the output** for your submission score

This baseline uses only tabular policy data with a simple Random Forest regressor.


In [ ]:
# 1. Initialize Client and Load Data

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from agentds import BenchmarkClient

# 🔑 REPLACE WITH YOUR CREDENTIALS
client = BenchmarkClient(
    api_key="your-api-key-here",        # Get from your team dashboard
    team_name="your-team-name-here"     # Your exact team name
)

# Load data from PVC paths
print("📂 Loading Insurance Challenge 2 data...")

# Load training and test policies (this challenge uses policies data only)
train_policies = pd.read_csv("/home/jovyan/shared/datasets/Insurance/train_policies_subset.csv")
test_policies = pd.read_csv("/home/jovyan/shared/datasets/Insurance/test_policies_subset.csv")

print(f"✅ Data loaded:")
print(f"   Train policies: {train_policies.shape}")
print(f"   Test policies: {test_policies.shape}")
print(f"   Columns: {list(train_policies.columns)}")


In [ ]:
# 2. Tabular-Only Baseline Model and Predictions

# Select specific numeric features from actual policy data
# From data inspection: HolderAge, AnnualMileage, LocationUrban, CreditScore
policy_features = ['HolderAge', 'AnnualMileage', 'LocationUrban', 'CreditScore']
print(f"📊 Using policy features: {policy_features}")

# Prepare training data
X_train = train_policies[policy_features].fillna(0)  # Simple imputation
y_train = train_policies['NextYearLoss']  # Target variable from policies

# Prepare test data
X_test = test_policies[policy_features].fillna(0)

# Train simple Random Forest baseline
print("🤖 Training Random Forest regressor...")
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Create submission file (format: PolicyID,NextYearLoss)
submission_df = pd.DataFrame({
    'PolicyID': test_policies['PolicyID'],
    'NextYearLoss': predictions
})

# Save predictions
submission_df.to_csv("insurance_challenge2_predictions.csv", index=False)
print(f"✅ Predictions saved: {submission_df.shape[0]} predictions")
print(f"   Preview: {submission_df.head(3)}")
print(f"   Loss range: ${predictions.min():.2f} to ${predictions.max():.2f}")


In [ ]:
# 3. Submit Predictions

# Submit predictions to the competition
print("🚀 Submitting predictions...")

try:
    result = client.submit_prediction("Insurance", 2, "insurance_challenge2_predictions.csv")
    
    if result['success']:
        print("✅ Submission successful!")
        print(f"   📊 Score: {result['score']:.4f}")
        print(f"   📏 Metric: {result['metric_name']}")
        print(f"   ✔️  Validation: {'Passed' if result['validation_passed'] else 'Failed'}")
    else:
        print("❌ Submission failed!")
        print(f"   Error details: {result.get('details', {}).get('validation_errors', 'Unknown error')}")
        
except Exception as e:
    print(f"💥 Submission error: {e}")
    print("🔧 Check your API key and team name are correct!")

print("\n🎯 Next steps:")
print("   1. Try incorporating relevant information outside this table!")
print("   2. Move on to Insurance Challenge 3!")
